# Value at Risk Exploration

## To what extent can Value at Risk serve as a reliable metric to gauge financial risk?
Many financial institutions have experienced huge losses over the past decade, due to fraud, bad management or poor advice. Regardless of the contributing factors they all strive to avoid repeating the same mistakes again. Therefore, institutions employ mathematical models to gauge financial risk.

Value at Risk (VaR) is an extremely popular mathematical model among corporate financial risk managers and an industry standard, to quantify the risk of financial assets. VaR is based on the probability of a possible loss over a specific time horizon in which this loss is expected to occur caused by adverse market movements (Simons, 2001).
VaR is extremely appealing, because it breaks risk down into a single number. Hence, it is also comprehensible by people without an extensive background in risk management, such as executives or customers.

However, as popularity of VaR increased criticism arose, too. The objective of this paper is to evaluate these critiques in order to determine whether Value at Risk can be used as a reliable metric to measure risk. There are countless different approaches to calculate VaR. In order to limit the scope of the paper the following methods shall be assessed: normal, hybrid and historical Value at Risk.
    
This paper will be based primarily on existing research. I’ll be using journals (from different databases) and technical literature. Existing approaches to calculate VaR and different perspectives will be explored. In order to test different theories I will apply these concepts and measure how well these three metrics perform against historical return information from a small sample of S&P 500 Stocks.

**Plan**

I'll begin the exploration by loading and analyzing a single dataset and write the necessary functions. I'll then work on writing loops to evaluate multiple models at the same time.
1. Import CSV File into Pandas Dataset
2. Initial Data Exploration
3. Preprocessing - calculating the Rate of Return
4. Normal Value at Risk
    * Calculate nVaR
    * Analyze the number, frequency and magnitude of exceedances
    * Compare and Contrast different nVaR confidence levels (99% & 95%)
5. Modified Value at Risk
    * Calculate mVaR
    * Analyze the number, frequency and magnitude of exceedances
    * Compare and Contrast different nVaR confidence levels (99% & 95%) 
6. Historical Value at Risk
    * Calculate hVaR
    * Analyze the number, frequency and magnitude of exceedances
    * Compare and Contrast different nVaR confidence levels (99%)  

## Data Loading
I'll begin my exploration with the analysis of the Amazon (AMZN) stock. I downloaded data on J